In [2]:
import glob, re, os, shutil, random
import pandas as pd
from pathlib import Path
import numpy as np

In [3]:
####################
#   Non-augmented  #
####################

df = pd.read_csv("csv\\label.csv", index_col= "No.")

TARGET_LIST = ['E/I', 'F/T', 'J/P', 'N/S']
TYPE_LIST = ['train', 'test']

df_MBTI_train = pd.DataFrame(columns = ['filename', 'MBTI'])
df_EI_train = pd.DataFrame(columns = ['filename', 'E/I']) 
df_FT_train = pd.DataFrame(columns = ['filename', 'F/T']) 
df_JP_train = pd.DataFrame(columns = ['filename', 'J/P'])
df_NS_train = pd.DataFrame(columns = ['filename', 'N/S']) 

df_MBTI_test = pd.DataFrame(columns = ['filename', 'MBTI'])
df_EI_test = pd.DataFrame(columns = ['filename', 'E/I']) 
df_FT_test = pd.DataFrame(columns = ['filename', 'F/T']) 
df_JP_test = pd.DataFrame(columns = ['filename', 'J/P'])
df_NS_test = pd.DataFrame(columns = ['filename', 'N/S']) 

for i in range(1, df.shape[0] + 1):
    file_name = f'{i}_*'
    for file in glob.glob(f'images\\original\\{file_name}'):
        E_I = df.loc[i, 'E/I']
        F_T = df.loc[i, 'F/T']
        J_P = df.loc[i, 'J/P']
        N_S = df.loc[i, 'N/S']

        SOURCE_PATH = f'{file}'
        if i % 5 == 0 : # TEST
            df_EI_test = df_EI_test.append({'filename':file, 'E/I':int(E_I == 'E')}, ignore_index=True)
            df_FT_test = df_FT_test.append({'filename':file, 'F/T':int(F_T == 'F')}, ignore_index=True)
            df_JP_test = df_JP_test.append({'filename':file, 'J/P':int(J_P == 'J')}, ignore_index=True)
            df_NS_test = df_NS_test.append({'filename':file, 'N/S':int(N_S == 'N')}, ignore_index=True)
            df_MBTI_test = df_MBTI_test.append({'filename':file, 'MBTI':E_I+N_S+F_T+J_P}, ignore_index=True)
        else : # TRAIN
            df_EI_train = df_EI_train.append({'filename':file, 'E/I':int(E_I == 'E')}, ignore_index=True)
            df_FT_train = df_FT_train.append({'filename':file, 'F/T':int(F_T == 'F')}, ignore_index=True)
            df_JP_train = df_JP_train.append({'filename':file, 'J/P':int(J_P == 'J')}, ignore_index=True)
            df_NS_train = df_NS_train.append({'filename':file, 'N/S':int(N_S == 'N')}, ignore_index=True)
            df_MBTI_train = df_MBTI_train.append({'filename':file, 'MBTI':E_I+N_S+F_T+J_P}, ignore_index=True)

df_EI_train.to_csv('csv\\original\\train_E_I.csv', index=False)
df_FT_train.to_csv('csv\\original\\train_F_T.csv', index=False)
df_JP_train.to_csv('csv\\original\\train_J_P.csv', index=False)
df_NS_train.to_csv('csv\\original\\train_N_S.csv', index=False)

df_EI_test.to_csv('csv\\original\\test_E_I.csv', index=False)
df_FT_test.to_csv('csv\\original\\test_F_T.csv', index=False)
df_JP_test.to_csv('csv\\original\\test_J_P.csv', index=False)
df_NS_test.to_csv('csv\\original\\test_N_S.csv', index=False)

df_MBTI_train.to_csv('csv\\original\\train_MBTI.csv', index=False)
df_MBTI_test.to_csv('csv\\original\\test_MBTI.csv', index=False)

In [4]:
####################
#     Augmented    #
####################

df = pd.read_csv("csv\\label.csv", index_col= "No.")

TARGET_LIST = ['E/I', 'F/T', 'J/P', 'N/S']
TYPE_LIST = ['train', 'test']

df_MBTI_train = pd.DataFrame(columns = ['filename', 'MBTI'])
df_EI_train = pd.DataFrame(columns = ['filename', 'E/I']) 
df_FT_train = pd.DataFrame(columns = ['filename', 'F/T']) 
df_JP_train = pd.DataFrame(columns = ['filename', 'J/P'])
df_NS_train = pd.DataFrame(columns = ['filename', 'N/S']) 

df_MBTI_test = pd.DataFrame(columns = ['filename', 'MBTI'])
df_EI_test = pd.DataFrame(columns = ['filename', 'E/I']) 
df_FT_test = pd.DataFrame(columns = ['filename', 'F/T']) 
df_JP_test = pd.DataFrame(columns = ['filename', 'J/P'])
df_NS_test = pd.DataFrame(columns = ['filename', 'N/S']) 

for i in range(1, df.shape[0] + 1):
    file_name = f'{i}_*'
    for file in glob.glob(f'images\\augmented\\{file_name}'):
        E_I = df.loc[i, 'E/I']
        F_T = df.loc[i, 'F/T']
        J_P = df.loc[i, 'J/P']
        N_S = df.loc[i, 'N/S']

        SOURCE_PATH = f'{file}'
        if i % 5 == 0 : # TEST
            df_EI_test = df_EI_test.append({'filename':file, 'E/I':int(E_I == 'E')}, ignore_index=True)
            df_FT_test = df_FT_test.append({'filename':file, 'F/T':int(F_T == 'F')}, ignore_index=True)
            df_JP_test = df_JP_test.append({'filename':file, 'J/P':int(J_P == 'J')}, ignore_index=True)
            df_NS_test = df_NS_test.append({'filename':file, 'N/S':int(N_S == 'N')}, ignore_index=True)
            df_MBTI_test = df_MBTI_test.append({'filename':file, 'MBTI':E_I+N_S+F_T+J_P}, ignore_index=True)
        else : # TRAIN
            df_EI_train = df_EI_train.append({'filename':file, 'E/I':int(E_I == 'E')}, ignore_index=True)
            df_FT_train = df_FT_train.append({'filename':file, 'F/T':int(F_T == 'F')}, ignore_index=True)
            df_JP_train = df_JP_train.append({'filename':file, 'J/P':int(J_P == 'J')}, ignore_index=True)
            df_NS_train = df_NS_train.append({'filename':file, 'N/S':int(N_S == 'N')}, ignore_index=True)
            df_MBTI_train = df_MBTI_train.append({'filename':file, 'MBTI':E_I+N_S+F_T+J_P}, ignore_index=True)

df_EI_train.to_csv('csv\\augmented\\train_E_I.csv', index=False)
df_FT_train.to_csv('csv\\augmented\\train_F_T.csv', index=False)
df_JP_train.to_csv('csv\\augmented\\train_J_P.csv', index=False)
df_NS_train.to_csv('csv\\augmented\\train_N_S.csv', index=False)

df_EI_test.to_csv('csv\\augmented\\test_E_I.csv', index=False)
df_FT_test.to_csv('csv\\augmented\\test_F_T.csv', index=False)
df_JP_test.to_csv('csv\\augmented\\test_J_P.csv', index=False)
df_NS_test.to_csv('csv\\augmented\\test_N_S.csv', index=False)

df_MBTI_train.to_csv('csv\\augmented\\train_MBTI.csv', index=False)
df_MBTI_test.to_csv('csv\\augmented\\test_MBTI.csv', index=False)